## PostgreSQL et JSON

In [1]:
import psycopg2
import psycopg2.extras

conn = psycopg2.connect(
    host="localhost",
    database="demo",
    user="demo_owner",
    password="OODBMS")

# query result as dict
c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

<div id="create-JSON-table"></div>
<h3>Création d'une table contenant des données JSON</h3>

In [2]:
data = [
  { "first_name": "Raymond", "last_name": "Deubaze",  "email": "raymond.deubaze@ec-lyon.fr" },
  { "first_name": "Jean",    "last_name": "Peuplu",   "email": "jpu@gmail.com" },
  { "first_name": "Alex",    "last_name": "Terrieur", "email": "atr1@gmail.com" },
  { "first_name": "Alain",   "last_name": "Terrieur", "email": "atr2@gmail.com" },
  { "first_name": "Anna",    "last_name": "Conda",    "email": "Anna.Conda@ec-lyon.fr" },
  { "first_name": "Ginette", "last_name": "Ringard",  "email": "ginette@wanadoo.fr" }
]

In [3]:
c.execute('DROP TABLE IF EXISTS json_people')

sql = '''CREATE TABLE json_people (
  id         SERIAL NOT NULL PRIMARY KEY,
  person     JSON NOT NULL
);'''
c.execute(sql)

conn.commit()

<div id="insert_json" style="font-size:120%">
La valeur donnée à un champ du type JSON doit être une chaîne de caractères conforme au standard JSON...
</div>

In [4]:
import json

sql = 'INSERT INTO json_people (person) VALUES (%s)'
for person in data:
    s = json.dumps(person)
    c.execute(sql,(s,))

conn.commit()

In [5]:
print(s)
print(type(s))

{"first_name": "Ginette", "last_name": "Ringard", "email": "ginette@wanadoo.fr"}
<class 'str'>


<div id="query_json"></div>

### Sélection de données JSON

<p style="font-size:120%">Le résultat d'une requête est par contre directement désérialisé par psycopg2.</p>
https://www.psycopg.org/docs/extras.html#json-adaptation

In [6]:
c.execute('SELECT person FROM json_people')
someone = c.fetchone()[0]
print(someone)
print(type(someone))

{'first_name': 'Raymond', 'last_name': 'Deubaze', 'email': 'raymond.deubaze@ec-lyon.fr'}
<class 'dict'>


<p id="query_json_attributes" style="font-size:120%">On peut requérir uniquement certains attributs JSON :</p>

In [7]:
c.execute("SELECT person ->> 'email' AS email FROM json_people")
r = c.fetchall()
for p in r:
    email = p['email']
    print(type(email),email)

<class 'str'> raymond.deubaze@ec-lyon.fr
<class 'str'> jpu@gmail.com
<class 'str'> atr1@gmail.com
<class 'str'> atr2@gmail.com
<class 'str'> Anna.Conda@ec-lyon.fr
<class 'str'> ginette@wanadoo.fr


<div id="json_where" style="font-size:120%">L'opérateur <code>->></code> peut également apparaître au sein de la clause WHERE :
</div>

In [8]:
c.execute("SELECT * FROM json_people WHERE person->>'first_name' = 'Anna'")
r = c.fetchone()
print(r)

[5, {'first_name': 'Anna', 'last_name': 'Conda', 'email': 'Anna.Conda@ec-lyon.fr'}]


<p style="font-size:120%">La différence entre les opérateurs <code>-></code> et <code>->></code> est gommée par le fait que psycopg2 désérialise automatiquement les chaînes JSON renvoyées par <code>-></code> :</p>

In [9]:
c.execute("SELECT person -> 'email' AS email FROM json_people")
r = c.fetchall()
for p in r:
    email = p['email']
    print(type(email),email)

<class 'str'> raymond.deubaze@ec-lyon.fr
<class 'str'> jpu@gmail.com
<class 'str'> atr1@gmail.com
<class 'str'> atr2@gmail.com
<class 'str'> Anna.Conda@ec-lyon.fr
<class 'str'> ginette@wanadoo.fr


<div id="json_embedded" style="font-size:120%">Quoique...</div>

In [10]:
raymond = {
    "first_name": "Raymond", "last_name": "Deubaze",
    "email": "raymond.deubaze@ec-lyon.fr",
    "office": {'building': 'H9', 'floor':0, 'room':'Aquitaine' }
}
sql = "UPDATE json_people SET person=%s WHERE person ->> 'first_name' = 'Raymond'"
c.execute(sql, (json.dumps(raymond),))
conn.commit()

<div id="psycopg2_str"></div>

In [11]:
c.execute("SELECT person ->> 'office' AS office FROM json_people WHERE person ->> 'office' IS NOT NULL")
r = c.fetchone()[0]
print(type(r), r)

<class 'str'> {"building": "H9", "floor": 0, "room": "Aquitaine"}


<div id="psycopg2_dict"></div>

In [12]:
c.execute("SELECT person -> 'office' AS office FROM json_people WHERE person ->> 'office' IS NOT NULL")
r = c.fetchone()[0]
print(type(r), r)

<class 'dict'> {'building': 'H9', 'floor': 0, 'room': 'Aquitaine'}


<div id="json_query_embedded"></div>

In [13]:
c.execute("SELECT person->'office'->>'room' FROM json_people WHERE person ->> 'office' IS NOT NULL")
print(c.fetchone()[0])

Aquitaine


<div id="json_functions"></div>

### Fonctions dédiées au contenu JSON

In [14]:
c.execute("SELECT json_object_keys(person) FROM json_people WHERE person ->> 'first_name' = 'Raymond'")
r = c.fetchall()
for p in r:
    print(p[0])

first_name
last_name
email
office


In [15]:
c.execute('''
    SELECT array_agg(o) FROM (
        SELECT json_object_keys(person) FROM json_people WHERE person ->> 'first_name' = 'Raymond'
    ) AS o
''')
r = c.fetchone()[0]
print(type(r),r)

<class 'str'> {(first_name),(last_name),(email),(office)}


In [16]:
c.execute("SELECT json_object_keys(person) FROM json_people WHERE person ->> 'first_name' = 'Raymond'")
keys = [v[0] for v in c.fetchall()]
print(keys)

['first_name', 'last_name', 'email', 'office']


<div id="json_typeof"></div>

In [17]:
c.execute("SELECT json_typeof(person -> 'office' -> 'floor') FROM json_people WHERE person ->> 'office' IS NOT NULL")
r = c.fetchone()[0]
print(r)

number


In [18]:
c.execute("SELECT person #>> '{office, building}' FROM json_people WHERE person ->> 'office' IS NOT NULL")
r = c.fetchone()[0]
print(r)

H9


<div id="to_json"></div>
### Génération d'un document JSON à partir du contenu d'une table

In [19]:
c.execute("SELECT array_agg(t) FROM (SELECT * from people) AS t")
print(c.fetchone()[0])

{"(1,Raymond,Deubaze,raymond.deubaze@ec-lyon.fr,t)","(2,Jean,Peuplu,jpu@gmail.com,f)","(3,Alex,Terrieur,atr1@gmail.com,f)","(4,Alain,Terrieur,atr2@gmail.com,f)","(5,Anna,Conda,Anna.Conda@ec-lyon.fr,t)","(6,Ginette,Ringard,ginette@wanadoo.fr,f)"}


In [20]:
c.execute('''
    SELECT to_json(o) FROM (
        SELECT array_agg(t) FROM (
            SELECT * from people    
        ) AS t
    ) AS o
''')
p = c.fetchone()[0]
print(type(p),p)

<class 'dict'> {'array_agg': [{'id': 1, 'first_name': 'Raymond', 'last_name': 'Deubaze', 'email': 'raymond.deubaze@ec-lyon.fr', 'centralien': True}, {'id': 2, 'first_name': 'Jean', 'last_name': 'Peuplu', 'email': 'jpu@gmail.com', 'centralien': False}, {'id': 3, 'first_name': 'Alex', 'last_name': 'Terrieur', 'email': 'atr1@gmail.com', 'centralien': False}, {'id': 4, 'first_name': 'Alain', 'last_name': 'Terrieur', 'email': 'atr2@gmail.com', 'centralien': False}, {'id': 5, 'first_name': 'Anna', 'last_name': 'Conda', 'email': 'Anna.Conda@ec-lyon.fr', 'centralien': True}, {'id': 6, 'first_name': 'Ginette', 'last_name': 'Ringard', 'email': 'ginette@wanadoo.fr', 'centralien': False}]}


In [21]:
c.execute('DROP TABLE IF EXISTS json_key_value')

sql = '''CREATE TABLE json_key_value (
  id      TEXT NOT NULL PRIMARY KEY,
  value   JSON NOT NULL
);'''
c.execute(sql)
conn.commit()

In [22]:
sql = '''INSERT INTO json_key_value (id,value) VALUES ( 'all', (
    SELECT to_json(o) FROM (
        SELECT array_agg(t) FROM (
            SELECT * from people
        ) AS t
    ) AS o
) -> 'array_agg' )'''
c.execute(sql)
conn.commit()

In [23]:
c.execute("SELECT value FROM json_key_value WHERE id = 'all'")
r = c.fetchone()[0]
print(type(r),r)

<class 'list'> [{'id': 1, 'first_name': 'Raymond', 'last_name': 'Deubaze', 'email': 'raymond.deubaze@ec-lyon.fr', 'centralien': True}, {'id': 2, 'first_name': 'Jean', 'last_name': 'Peuplu', 'email': 'jpu@gmail.com', 'centralien': False}, {'id': 3, 'first_name': 'Alex', 'last_name': 'Terrieur', 'email': 'atr1@gmail.com', 'centralien': False}, {'id': 4, 'first_name': 'Alain', 'last_name': 'Terrieur', 'email': 'atr2@gmail.com', 'centralien': False}, {'id': 5, 'first_name': 'Anna', 'last_name': 'Conda', 'email': 'Anna.Conda@ec-lyon.fr', 'centralien': True}, {'id': 6, 'first_name': 'Ginette', 'last_name': 'Ringard', 'email': 'ginette@wanadoo.fr', 'centralien': False}]


<div id="select_json_list"></div>

### Sélection dans une liste JSON

In [24]:
c.execute("SELECT value -> 5 -> 'email' FROM json_key_value WHERE id = 'all'")
print(c.fetchone()[0])

ginette@wanadoo.fr


<div id="json_to_recordset"></div>
### Transformation d'une liste JSON en enregistrements

In [25]:
sql = '''SELECT person.* FROM json_to_recordset(
    (SELECT value FROM json_key_value WHERE id='all'))
    AS person(id INT, first_name TEXT, last_name TEXT, email TEXT, centralien BOOLEAN)
'''
c.execute(sql)
for p in c.fetchall(): print(p)

[1, 'Raymond', 'Deubaze', 'raymond.deubaze@ec-lyon.fr', True]
[2, 'Jean', 'Peuplu', 'jpu@gmail.com', False]
[3, 'Alex', 'Terrieur', 'atr1@gmail.com', False]
[4, 'Alain', 'Terrieur', 'atr2@gmail.com', False]
[5, 'Anna', 'Conda', 'Anna.Conda@ec-lyon.fr', True]
[6, 'Ginette', 'Ringard', 'ginette@wanadoo.fr', False]


In [26]:
c.execute('DROP TABLE IF EXISTS temp_people')

sql = '''CREATE TABLE temp_people (
  id         SERIAL PRIMARY KEY,
  first_name TEXT,
  last_name  TEXT,
  email      TEXT,
  centralien BOOLEAN
);'''
c.execute(sql)

conn.commit()

In [27]:
c.execute('''
  INSERT INTO temp_people(id, first_name, last_name, email, centralien)
    SELECT person.* FROM json_to_recordset(
      (SELECT value FROM json_key_value WHERE id='all')
    )
    AS person(id INT, first_name TEXT, last_name TEXT, email TEXT, centralien BOOLEAN)
''')
conn.commit()

In [28]:
c.execute("SELECT * FROM temp_people")
for p in c.fetchall(): print(p)

[1, 'Raymond', 'Deubaze', 'raymond.deubaze@ec-lyon.fr', True]
[2, 'Jean', 'Peuplu', 'jpu@gmail.com', False]
[3, 'Alex', 'Terrieur', 'atr1@gmail.com', False]
[4, 'Alain', 'Terrieur', 'atr2@gmail.com', False]
[5, 'Anna', 'Conda', 'Anna.Conda@ec-lyon.fr', True]
[6, 'Ginette', 'Ringard', 'ginette@wanadoo.fr', False]


In [29]:
conn.rollback()

<div id="json_update"></div>
### Modification d'un document JSON

<p style="font-size:120%">
La syntaxe intuitive ne fonctionne pas :
</p>

In [30]:
sql = '''
    UPDATE json_people
    SET person->'office'->'room'=%s
    WHERE person->>'first_name'='Raymond'
'''
try:
    c.execute(sql, ('Bretagne',))
    conn.commit()
except psycopg2.errors.SyntaxError as e:
    print(e)
    conn.rollback()

ERREUR:  erreur de syntaxe sur ou près de « -> »
LINE 3:     SET person->'office'->'room'='Bretagne'
                      ^



<p style="font-size:120%" id="jsonb_set">
Ajout ou modification d'un champ dans un document JSON
</p>
<p>Le principe de la méthode repose sur la fonction <code>jsonb_set</code> qui permet d'insrer/modifier un attribut au sein d'un document JSON.</p>
<p>Malheureusement cette fonction traite une donnée au format jsonb et nécessite donc une conversion avant/après depuis et vers le format json quand on désire modifier un champ de ce type.</p>

In [31]:
sql = '''
    SELECT to_json(jsonb_set(to_jsonb(person), '{office, room}', '"Aquitaine"'))
    FROM json_people
    WHERE person->>'office' IS NOT NULL
'''
c.execute(sql)
for p in c.fetchall(): print(p)

[{'email': 'raymond.deubaze@ec-lyon.fr', 'office': {'room': 'Aquitaine', 'floor': 0, 'building': 'H9'}, 'last_name': 'Deubaze', 'first_name': 'Raymond'}]


Il ne reste plus qu'à insérer cette opération dans une requête UPDATE :

In [32]:
# ajout d'un bureau à Ginette
sql = '''
    UPDATE json_people SET person = result.value
    FROM (
        SELECT to_json(jsonb_set(to_jsonb(person), '{office}', %s)) AS value
        FROM json_people
    ) AS result
    WHERE result.value->>'first_name' = json_people.person->>'first_name'
    AND json_people.person->>'first_name' = %s
'''
office = {'building': 'H9', 'floor':0, 'room':'Aquitaine' }

c.execute(sql,(json.dumps(office),'Ginette'))
conn.commit()

In [33]:
c.execute('SELECT to_jsonb(person) from json_people')
for p in c.fetchall(): print(p)

[{'email': 'jpu@gmail.com', 'last_name': 'Peuplu', 'first_name': 'Jean'}]
[{'email': 'atr1@gmail.com', 'last_name': 'Terrieur', 'first_name': 'Alex'}]
[{'email': 'atr2@gmail.com', 'last_name': 'Terrieur', 'first_name': 'Alain'}]
[{'email': 'Anna.Conda@ec-lyon.fr', 'last_name': 'Conda', 'first_name': 'Anna'}]
[{'email': 'raymond.deubaze@ec-lyon.fr', 'office': {'room': 'Aquitaine', 'floor': 0, 'building': 'H9'}, 'last_name': 'Deubaze', 'first_name': 'Raymond'}]
[{'email': 'ginette@wanadoo.fr', 'office': {'room': 'Aquitaine', 'floor': 0, 'building': 'H9'}, 'last_name': 'Ringard', 'first_name': 'Ginette'}]


In [34]:
# modification de la pièce du bureau de Raymond
sql = '''
    UPDATE json_people SET person = result.value
    FROM (
        SELECT to_json(jsonb_set(to_jsonb(person), '{office, room}', %s)) AS value
        FROM json_people
    ) AS result
    WHERE result.value->>'first_name' = json_people.person->>'first_name'
    AND json_people.person->>'first_name' = %s
'''
c.execute(sql,(json.dumps('Chausey'),'Raymond'))
conn.commit()

In [35]:
c.execute("SELECT to_jsonb(person) from json_people")
for p in c.fetchall(): print(p)

[{'email': 'jpu@gmail.com', 'last_name': 'Peuplu', 'first_name': 'Jean'}]
[{'email': 'atr1@gmail.com', 'last_name': 'Terrieur', 'first_name': 'Alex'}]
[{'email': 'atr2@gmail.com', 'last_name': 'Terrieur', 'first_name': 'Alain'}]
[{'email': 'Anna.Conda@ec-lyon.fr', 'last_name': 'Conda', 'first_name': 'Anna'}]
[{'email': 'ginette@wanadoo.fr', 'office': {'room': 'Aquitaine', 'floor': 0, 'building': 'H9'}, 'last_name': 'Ringard', 'first_name': 'Ginette'}]
[{'email': 'raymond.deubaze@ec-lyon.fr', 'office': {'room': 'Chausey', 'floor': 0, 'building': 'H9'}, 'last_name': 'Deubaze', 'first_name': 'Raymond'}]


<p id="jsonb_set_lax" style="font-size:120%">La suppression d'un attribut JSON peut s'effectuer à l'aide de la méthode <code>jsonb_set_lax</code>.
</p>
<div>Noter l'argument 'delete_key' et la valeur None (NULL en SQL) donnée à la clé 'office'...
</div>

In [36]:
# suppression du bureau de Ginette
sql = '''
    UPDATE json_people SET person = result.value
    FROM (
        SELECT to_json(jsonb_set_lax(to_jsonb(person), '{office}', %s, FALSE, 'delete_key')) AS value
        FROM json_people
    ) AS result
    WHERE result.value->>'first_name' = json_people.person->>'first_name'
    AND json_people.person->>'first_name' = %s
'''
office = None

c.execute(sql,(json.dumps(office),'Ginette'))
conn.commit()

In [37]:
c.execute("SELECT to_jsonb(person) from json_people")
for p in c.fetchall(): print(p)

[{'email': 'jpu@gmail.com', 'last_name': 'Peuplu', 'first_name': 'Jean'}]
[{'email': 'atr1@gmail.com', 'last_name': 'Terrieur', 'first_name': 'Alex'}]
[{'email': 'atr2@gmail.com', 'last_name': 'Terrieur', 'first_name': 'Alain'}]
[{'email': 'Anna.Conda@ec-lyon.fr', 'last_name': 'Conda', 'first_name': 'Anna'}]
[{'email': 'raymond.deubaze@ec-lyon.fr', 'office': {'room': 'Chausey', 'floor': 0, 'building': 'H9'}, 'last_name': 'Deubaze', 'first_name': 'Raymond'}]
[{'email': 'ginette@wanadoo.fr', 'office': None, 'last_name': 'Ringard', 'first_name': 'Ginette'}]


<div id="JSONPath"></div>

### JSONPath

In [44]:
c.execute("SELECT jsonb_path_exists(to_jsonb(person), '$.office') FROM json_people WHERE person->>'first_name'='Raymond'")
print(c.fetchone()[0])

True


In [46]:
c.execute("SELECT jsonb_path_query(to_jsonb(person), '$.office') FROM json_people WHERE person->>'first_name'='Raymond'")
r = c.fetchone()[0]
print(type(r),r)

<class 'dict'> {'room': 'Chausey', 'floor': 0, 'building': 'H9'}


In [52]:
# pas besoin de json_value

c.execute("SELECT jsonb_path_query(to_jsonb(person), '$.first_name') FROM json_people")
r = c.fetchone()[0]
print(type(r),r)

c.execute("SELECT jsonb_path_query(to_jsonb(person), '$.office.floor') FROM json_people")
for p in c.fetchall():
    print(type(p[0]),p[0])

<class 'str'> Jean
<class 'int'> 0


In [58]:
# ajout d'une liste d'amis à Raymond pour tester [n]
sql = '''
    UPDATE json_people SET person = result.value
    FROM (
        SELECT to_json(jsonb_set(to_jsonb(person), '{friends}', %s)) AS value
        FROM json_people
    ) AS result
    WHERE result.value->>'first_name' = json_people.person->>'first_name'
    AND json_people.person->>'first_name' = %s
'''
friends = [
    { 'first_name': 'Jean', 'last_name': 'Bombeur'},
    { 'first_name': 'Elsa', 'last_name': 'Plique'}
]

c.execute(sql,(json.dumps(friends),'Raymond'))
conn.commit()

<div id="json_path_query_list"></div>

In [59]:
c.execute('''
  SELECT jsonb_path_query(to_jsonb(person), '$.friends[1].first_name')
  FROM json_people WHERE person->>'first_name'='Raymond'
''')
r = c.fetchone()[0]
print(type(r),r)

<class 'str'> Elsa


In [90]:
# Ajout d'une information booléenne aux personnes
sql = '''
    UPDATE json_people SET person = result.value
    FROM (
        SELECT to_json(jsonb_set(to_jsonb(person), '{centralien}', to_jsonb(position('ec-lyon' in person->>'email') <> 0))) AS value
        FROM json_people
    ) AS result
    WHERE result.value->>'first_name' = json_people.person->>'first_name'
'''
c.execute(sql)
conn.commit()

<div id="json_path_filter"></div>

In [115]:
c.execute('''
  SELECT jsonb_path_query(to_jsonb(person), '($ ? (@.centralien == true)).first_name') FROM json_people
''')
for p in c.fetchall():
    print(type(p[0]),p[0])

<class 'str'> Anna
<class 'str'> Raymond


In [125]:
c.execute('''
  SELECT jsonb_path_query(to_jsonb(person), '($ ? (@.first_name == "Raymond")).office') FROM json_people
''')
for p in c.fetchall():
    print(type(p[0]),p[0])

<class 'dict'> {'room': 'Chausey', 'floor': 0, 'building': 'H9'}


In [130]:
c.execute('''
  SELECT jsonb_path_query(to_jsonb(person), '$.friends') FROM json_people
''')
for p in c.fetchall():
    print(type(p[0]),p[0])

<class 'list'> [{'last_name': 'Bombeur', 'first_name': 'Jean'}, {'last_name': 'Plique', 'first_name': 'Elsa'}]


<div id="json_path_joker"></div>

In [131]:
c.execute('''
  SELECT jsonb_path_query(to_jsonb(person), '$.friends[*]') FROM json_people
''')
for p in c.fetchall():
    print(type(p[0]),p[0])

<class 'dict'> {'last_name': 'Bombeur', 'first_name': 'Jean'}
<class 'dict'> {'last_name': 'Plique', 'first_name': 'Elsa'}


In [132]:
c.execute('''
  SELECT jsonb_path_query(to_jsonb(person), '$.friends[*].*') FROM json_people
''')
for p in c.fetchall():
    print(type(p[0]),p[0])

<class 'str'> Bombeur
<class 'str'> Jean
<class 'str'> Plique
<class 'str'> Elsa


<div id="json_path_joker_filter"></div>

In [138]:
c.execute('''
  SELECT jsonb_path_query(to_jsonb(person), '$.friends?(@.first_name == "Jean")[*]') FROM json_people
''')
for p in c.fetchall():
    print(type(p[0]),p[0])

<class 'dict'> {'last_name': 'Bombeur', 'first_name': 'Jean'}
